In [1]:
import pandas as pd
import numpy as np

In [137]:
TRAINING_DATA_FILE="SpamData/02_Training/train_data.txt"
TEST_DATA_FILE="SpamData/02_Training/test_data.txt"
TOKEN_SPAM_PROB_FILE="SpamData/03_Testing/prob-spam.txt"
TOKEN_HAM_PROB_FILE="SpamData/03_Testing/prob-ham.txt"
TOKEN_ALL_PROB_FILE="SpamData/03_Testing/prob-all-tokens.txt"
TEST_FEATURE_MATRIX="SpamData/03_Testing/test-features.txt"
TEST_TARGET_FILE="SpamData/03_Testing/test-target.txt"
VOCAB_SIZE=2500

In [3]:
sparse_train_data=np.loadtxt(TRAINING_DATA_FILE,delimiter=" ",dtype=int)
sparse_train_data

array([[   0,    2,    1,    1],
       [   0,    3,    1,    2],
       [   0,    4,    1,    1],
       ...,
       [5795, 2272,    0,    1],
       [5795, 2342,    0,    1],
       [5795, 2370,    0,    1]])

In [4]:
sparse_test_data=np.loadtxt(TEST_DATA_FILE,delimiter=" ",dtype=int)
sparse_test_data

array([[   8,    2,    1,    1],
       [   8,    3,    1,    4],
       [   8,    4,    1,    2],
       ...,
       [5793, 2374,    0,    4],
       [5793, 2403,    0,    3],
       [5793, 2491,    0,    1]])

In [6]:
print("Nr of rows in training data",sparse_train_data.shape[0])
print("Nr of rows in testing data",sparse_test_data.shape[0])

Nr of rows in training data 258360
Nr of rows in testing data 117776


In [12]:
print("Nr of emails in training data",np.unique(sparse_train_data[:,0]).size)
print("Nr of emails in test data",np.unique(sparse_test_data[:,0]).size)

Nr of emails in training data 4013
Nr of emails in test data 1724


In [14]:
column_names=["DOC_ID"]+["CATEGORY"]+list(range(0,VOCAB_SIZE))
column_names[:6]

['DOC_ID', 'CATEGORY', 0, 1, 2, 3]

In [15]:
len(column_names)

2502

In [17]:
index_names=np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [21]:
full_train_data=pd.DataFrame(index=index_names,columns=column_names)
full_train_data.fillna(value=0,inplace=True)

In [22]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
def make_full_metrix(sparse_matrix,nr_words,doc_idx=0,word_idx=1,cat_idx=2,freq_idx=3):
    column_names=["DOC_ID"]+["CATEGORY"]+list(range(0,VOCAB_SIZE))
    doc_id_names=np.unique(sparse_matrix[:,0])
    full_matrix=pd.DataFrame(index=doc_id_names,columns=column_names)
    full_matrix.fillna(value=0,inplace=True)
    for i in range(sparse_matrix.shape[0]):
        doc_nr=sparse_matrix[i][doc_idx]
        word_id=sparse_matrix[i][word_idx]
        label=sparse_matrix[i][cat_idx]
        occurence=sparse_matrix[i][freq_idx]
        full_matrix.at[doc_nr,"DOC_ID"]=doc_nr
        full_matrix.at[doc_nr,"CATEGORY"]=label
        full_matrix.at[doc_nr,word_id]=occurence
    full_matrix.set_index("DOC_ID",inplace=True)
    return full_matrix

In [25]:
full_train_data=make_full_metrix(sparse_train_data,VOCAB_SIZE)

In [26]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Training the Naive Bayes Model
### Calculating the Probability of the Spam

In [45]:
# total_spam=full_train_data[full_train_data.CATEGORY==1].index.values.size
# total_mail=full_train_data.index.values.size
# (total_spam/total_mail)*100

31.098928482432097

In [49]:
prob_spam=full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
print("Probability of the spam:",prob_spam)

Probability of the spam: 0.310989284824321


In [50]:
full_train_features=full_train_data.loc[:,full_train_data.columns!="CATEGORY"]
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
email_lengths=full_train_features.sum(axis=1)
email_lengths

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
5789     18
5790     72
5791     46
5794     57
5795    264
Length: 4013, dtype: int64

In [52]:
total_wc=email_lengths.sum()
total_wc

429241

In [107]:
spam_length=email_lengths[full_train_data.CATEGORY==1]
ham_length=email_lengths[full_train_data.CATEGORY==0]
# spam_length+ham_length
# spam_length

In [102]:
spam_wc=spam_length.sum()
ham_wc=ham_length.sum()

In [96]:
email_lengths[max(email_lengths)==email_lengths]

DOC_ID
5775    6873
dtype: int64

In [105]:
print("Average number of the spam email {:.0f}".format(spam_wc/spam_length.shape[0]))
print("Average number of the ham email {:.0f}".format(ham_wc/ham_length.shape[0]))

Average number of the spam email 141
Average number of the ham email 91


In [108]:
train_spam_token=full_train_features.loc[full_train_data.CATEGORY==1]
train_spam_token.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
train_spam_token.shape

(1248, 2500)

In [118]:
summed_spam_token=train_spam_token.sum(axis=0)+1
summed_spam_token[min(summed_spam_token)==summed_spam_token]

266     1
294     1
371     1
398     1
443     1
       ..
2438    1
2443    1
2462    1
2465    1
2492    1
Length: 156, dtype: int64

In [122]:
train_ham_token=full_train_features.loc[full_train_data.CATEGORY==0]
summed_ham_token=train_ham_token.sum(axis=0)+1          # Laplac Smoothing technique
summed_ham_token[min(summed_ham_token)==summed_ham_token]

638     1
872     1
891     1
894     1
939     1
1207    1
1209    1
1231    1
1297    1
1299    1
1423    1
1478    1
1740    1
1787    1
1851    1
1892    1
1942    1
2111    1
2112    1
2123    1
2186    1
2226    1
2259    1
2289    1
2307    1
2335    1
2388    1
2392    1
2398    1
2410    1
2455    1
2457    1
2467    1
dtype: int64

## P(Token|Spam)

In [127]:
prob_token_spam=summed_spam_token/(spam_wc+VOCAB_SIZE)
prob_token_spam[:5]

0    0.012184
1    0.005228
2    0.006805
3    0.011307
4    0.006816
dtype: float64

## P(Token|ham)

In [128]:
prob_token_ham=summed_ham_token/(ham_wc+VOCAB_SIZE)
prob_token_ham[:5]

0    0.021472
1    0.010141
2    0.008007
3    0.003673
4    0.006311
dtype: float64

## P(Token)

In [131]:
prob_tokens_all=full_train_features.sum(axis=0)/total_wc
prob_tokens_all[:5]

0    0.017848
1    0.008208
2    0.007595
3    0.006891
4    0.006591
dtype: float64

In [136]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_token_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_token_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_tokens_all) 


In [139]:
sparse_test_data.shape

(117776, 4)

In [140]:
full_test_data=make_full_metrix(sparse_test_data,VOCAB_SIZE)

In [142]:
X_test=full_test_data.loc[:,full_test_data.columns!="CATEGORY"]
y_test=full_test_data.CATEGORY

In [143]:
np.savetxt(TEST_FEATURE_MATRIX,X_test)
np.savetxt(TEST_TARGET_FILE,y_test)